<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 7


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----
Создать базовый класс BankAccount в C#, который будет представлять
информацию об учетных записях в банке. На основе этого класса разработать 2-3
производных класса, демонстрирующих принципы наследования и полиморфизма.
В каждом из классов должны быть реализованы новые атрибуты и методы, а также
переопределены некоторые методы базового класса для демонстрации
полиморфизма. 

<b>Требования к базовому классу BankAccount:</b>
<ul>
    <li>Атрибуты: Номер счета (AccountNumber), Баланс (Balance), Тип счет (AccountType).</li>
    <li>Методы:</li>
    <ul>
        <li>GetInfo(): метод для получения информации о счете в виде строки.</li>
        <li>Deposit(decimal amount): метод для внесения денег на счет.</li>
        <li>Withdraw(decimal amount): метод для снятия денег со счета.</li>
    </ul>
</ul>

<b>Требования к производным классам:</b>

<ol>
    <li>Сберегательный счет (SavingsAccount): Должен содержать дополнительные атрибуты, такие как Процентная ставка (InterestRate). Метод Deposit() должен быть переопределен для добавления процентов к сумме вклада при внесении денег на счет.</li>
    <li>Текущий счет (CheckingAccount): Должен содержать дополнительные атрибуты, такие как Лимит овердрафта (OverdraftLimit). Метод Withdra () должен быть переопределен для проверки и применения лимита овердрафта при снятии денег со счета.</li>
    <li>Инвестиционный счет (InvestmentAccount) (если требуется третий класс): Должен содержать дополнительные атрибуты, такие как Список активов (AssetsList). Метод GetInfo() должен быть переопределен для включения информации о списках активов в описании счета.</li>
</ol>

Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;

public interface IBankOperations
{
    void Deposit(decimal amount);
    void Withdraw(decimal amount);
    string GetInfo();
}

// Делегат возвращает строку, чтобы соответствовать методу GetInfo
public delegate string DisplayDelegate();

public class BankAccount : IBankOperations
{
    public string AccountNumber { get; set; }
    public decimal Balance { get; protected set; }
    public string AccountType { get; protected set; }

    // Событие изменения баланса
    public event Action<decimal> BalanceChanged;

    public BankAccount(string accountNumber, decimal initialBalance, string accountType)
    {
        AccountNumber = accountNumber;
        Balance = initialBalance;
        AccountType = accountType;
    }

    public virtual void Deposit(decimal amount)
    {
        if (amount <= 0)
            throw new ArgumentException("Сумма пополнения должна быть больше 0.");
        Balance += amount;
        BalanceChanged?.Invoke(Balance);
    }

    public virtual void Withdraw(decimal amount)
    {
        if (amount > Balance)
            throw new InvalidOperationException("Недостаточно средств.");
        Balance -= amount;
        BalanceChanged?.Invoke(Balance);
    }

    public virtual string GetInfo()
    {
        return $"Номер счета: {AccountNumber}, Баланс: {Balance:C}, Тип счета: {AccountType}";
    }
}

public class SavingsAccount : BankAccount
{
    public decimal InterestRate { get; set; }

    public SavingsAccount(string accountNumber, decimal initialBalance, string accountType, decimal interestRate)
        : base(accountNumber, initialBalance, accountType)
    {
        InterestRate = interestRate;
    }

    public override void Deposit(decimal amount)
    {
        base.Deposit(amount);
        Balance += Balance * (InterestRate / 100); // Начисление процентов
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $", Процентная ставка: {InterestRate}%";
    }
}

public class SecureDepositAccount : SavingsAccount, IBankOperations
{
    public bool IsAccountBlocked { get; private set; }

    public SecureDepositAccount(string accountNumber, decimal initialBalance, string accountType, decimal interestRate)
        : base(accountNumber, initialBalance, accountType, interestRate)
    {
        IsAccountBlocked = false;
    }

    void IBankOperations.Deposit(decimal amount)
    {
        if (IsAccountBlocked)
        {
            throw new InvalidOperationException("Счет заблокирован. Внесение средств невозможно.");
        }
        base.Deposit(amount);
    }

    void IBankOperations.Withdraw(decimal amount)
    {
        if (IsAccountBlocked)
        {
            throw new InvalidOperationException("Счет заблокирован. Снятие средств невозможно.");
        }
        base.Withdraw(amount);
    }

    string IBankOperations.GetInfo()
    {
        string blockedStatus = IsAccountBlocked ? "Заблокирован" : "Активен";
        return base.GetInfo() + $", Статус счета: {blockedStatus}";
    }

    public void BlockAccount()
    {
        IsAccountBlocked = true;
        Console.WriteLine($"Счет {AccountNumber} заблокирован.");
    }
}

public class AccountManager
{
    private readonly IBankOperations _bankOperations;

    public AccountManager(IBankOperations bankOperations)
    {
        _bankOperations = bankOperations;
    }

    public void MakeDeposit(decimal amount)
    {
        _bankOperations.Deposit(amount);
        Console.WriteLine($"Пополнение в размере {amount:C} успешно выполнено.");
    }

    public void MakeWithdrawal(decimal amount)
    {
        _bankOperations.Withdraw(amount);
        Console.WriteLine($"Снятие в размере {amount:C} успешно выполнено.");
    }

    public void PrintAccountInfo()
    {
        Console.WriteLine(_bankOperations.GetInfo());
    }
}


        // Создаем счета
        var secureAccount = new SecureDepositAccount("98765", 2000, "Депозит", 2.5m);
        var savingsAccount = new SavingsAccount("12345", 1500, "Сберегательный", 3.0m);

        // Подписываемся на события изменения баланса
        secureAccount.BalanceChanged += newBalance => Console.WriteLine($"SecureAccount новый баланс: {newBalance:C}");
        savingsAccount.BalanceChanged += newBalance => Console.WriteLine($"SavingsAccount новый баланс: {newBalance:C}");

        // Используем делегат для отображения информации
        DisplayDelegate displaySecure = secureAccount.GetInfo;
        DisplayDelegate displaySavings = savingsAccount.GetInfo;

        Console.WriteLine("Информация о счетах:");
        Console.WriteLine(displaySecure.Invoke()); // Вызов делегата
        Console.WriteLine(displaySavings.Invoke());

        var manager = new AccountManager(savingsAccount);
        manager.MakeDeposit(500);
        manager.MakeWithdrawal(200);
        manager.PrintAccountInfo();

        // Лямбда-выражения для фильтрации
        List<BankAccount> accounts = new List<BankAccount> { secureAccount, savingsAccount };
        var highBalanceAccounts = accounts.FindAll(acc => acc.Balance > 1000);

        Console.WriteLine("\nСчета с балансом более 1000:");
        foreach (var acc in highBalanceAccounts)
        {
            Console.WriteLine(acc.GetInfo());
        }



Информация о счетах:
Номер счета: 98765, Баланс: 2 000,00 ¤, Тип счета: Депозит, Процентная ставка: 2,5%
Номер счета: 12345, Баланс: 1 500,00 ¤, Тип счета: Сберегательный, Процентная ставка: 3,0%
SavingsAccount новый баланс: 2 000,00 ¤
Пополнение в размере 500,00 ¤ успешно выполнено.
SavingsAccount новый баланс: 1 860,00 ¤
Снятие в размере 200,00 ¤ успешно выполнено.
Номер счета: 12345, Баланс: 1 860,00 ¤, Тип счета: Сберегательный, Процентная ставка: 3,0%

Счета с балансом более 1000:
Номер счета: 98765, Баланс: 2 000,00 ¤, Тип счета: Депозит, Процентная ставка: 2,5%
Номер счета: 12345, Баланс: 1 860,00 ¤, Тип счета: Сберегательный, Процентная ставка: 3,0%


In [ ]:
IBankOperations secureAccount = new SecureDepositAccount("98765", 2000, "Сберегательный", 3);

AccountManager manager = new AccountManager(secureAccount);

manager.MakeDeposit(500);
manager.PrintAccountInfo();

manager.MakeWithdrawal(300);
manager.PrintAccountInfo();

Пополнение в размере 500,00 ¤ успешно выполнено.
Номер счета: 98765, Баланс: 2 575,00 ¤, Тип счета: Сберегательный, Процентная ставка: 3%, Статус счета: Активен
Снятие в размере 300,00 ¤ успешно выполнено.
Номер счета: 98765, Баланс: 2 275,00 ¤, Тип счета: Сберегательный, Процентная ставка: 3%, Статус счета: Активен


In [2]:

AccountStorage<BankAccount> storage = new AccountStorage<BankAccount>();
storage.AddAccount(savingsAccount);
storage.AddAccount(checkingAccount);
storage.AddAccount(investmentAccount);


storage.PrintAllAccountsInfo();
storage.ShowCount();


In [69]:
BonusSavingsAccount bonusSavingsAccount = new BonusSavingsAccount(
"987654", 6000, "Сберегательный", 3, 2); 

Console.WriteLine(bonusSavingsAccount.GetInfo());
bonusSavingsAccount.ApplyBonus();
Console.WriteLine(bonusSavingsAccount.GetInfo());

In [62]:

SecureDepositAccount secureAccount = new SecureDepositAccount("98765", 2000, "Депозит", 2.5m);
Console.WriteLine(secureAccount.GetInfo());
secureAccount.Withdraw(1901);
secureAccount.CheckSuspiciousActivity();
Console.WriteLine(secureAccount.GetInfo());



In [81]:
MortgageAccount mortgageAccount = new MortgageAccount("98765", 80000,"Ипотечный",4, 360);
Console.WriteLine(mortgageAccount.GetInfo());

In [137]:
savingsAccount.Transfer(checkingAccount, 100);
Console.WriteLine(savingsAccount.GetInfo());
Console.WriteLine(checkingAccount.GetInfo());